<a href="https://colab.research.google.com/github/RicoStaedeli/NLP2025_CQG/blob/main/2_Baseline_CQS_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline Predictions
In this file we generate the baseline predictions

## Setup

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import json
import logging
import tqdm
import re
import torch
from getpass import getpass
from google.colab import userdata, drive
import os

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
token = userdata.get('GITHUB')
repo_url = f"https://{token}@github.com/RicoStaedeli/NLP2025_CQG.git"

!git clone {repo_url}

Cloning into 'NLP2025_CQG'...
remote: Enumerating objects: 351, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 351 (delta 37), reused 30 (delta 26), pack-reused 301 (from 1)
Receiving objects: 100% (351/351), 24.22 MiB | 9.21 MiB/s, done.
Resolving deltas: 100% (143/143), done.


In [4]:
os.chdir("NLP2025_CQG")
!ls

1_Preprocessing.ipynb		   Doc
2a_Baseline_Evaluation.ipynb	   Evaluation
2_Baseline_CQS_generation.ipynb    LICENSE
3a_Finetuned_CQS_generation.ipynb  Logs
3b_Finetune_Evaluation.ipynb	   README.md
3_Training.ipynb		   requirements.txt
4_Evaluation_Analytics.ipynb	   Training
Data				   Utils
Development


In [5]:
################################################################################
#######################   PATH VARIABLES        ################################
################################################################################

test_dataset_path = "Data/Processed/test.json"
model_path_llama = "/content/drive/MyDrive/HSG/NLP/Project NLP/Models/Meta-Llama-3.1-8B-Instruct"
model_path_qwen = "/content/drive/MyDrive/HSG/NLP/Project NLP/Models/Qwen2.5-7B-Instruct"
results_path = "Evaluation/Results/"
log_path = "Logs/2_baseline_predictions.log"

################################################################################
#######################   STATIC VARIABLES      ################################
################################################################################

# Setup logger manually
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create file handler (only if not already added)
if not logger.handlers:
    fh = logging.FileHandler(log_path)
    fh.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    logger.addHandler(fh)

# Detect device
device = torch.device(
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)

# Log the device info
logger.info("--------  Start with Baseline Predictions  -------------")
logger.info(f'Device selected: {device}')

INFO:__main__:--------  Start with Baseline Predictions  -------------
INFO:__main__:Device selected: cuda


## Zero Shot prompting
In this section we genererate critical questions with different pretrained vanilla models. We use this generated questions as a baseline to compare it against our results. The following models were used to generate the baseline results:
- LLama 3.1 8B Instruct
- Qwen 2.5 7B Instruct

In [6]:
models = [
    {
        "name": "llama",
        "model_id": model_path_llama,
        "output_file": results_path + "results_zeroshot_llama_3.1-8B-instruct.json",
    },
    {
        "name": "qwen",
        "model_id": model_path_qwen,
        "output_file": results_path + "results_zeroshot_qwen2.5-7b-instruction.json",
    },
]

## Generate critical Questions

In [7]:
batch_size = 8  # You can adjust this based on your GPU memory

def structure_output(whole_text):
    cqs_list = whole_text.split('\n')
    final = []
    valid = []
    not_valid = []
    for cq in cqs_list:
        if re.match(r'.*\?(\")?( )?(\([a-zA-Z0-9\.\'-\,\? ]*\))?([a-zA-Z \.,\"\']*)?(\")?$', cq):
            valid.append(cq)
        else:
            not_valid.append(cq)

    still_not_valid = []
    for text in not_valid:
        new_cqs = re.split(r'\?\"', text + 'end')
        if len(new_cqs) > 1:
            for cq in new_cqs[:-1]:
                valid.append(cq + '?"')
        else:
            still_not_valid.append(text)

    for i, cq in enumerate(valid):
        occurrence = re.search(r'[A-Z]', cq)
        if occurrence:
            final.append(cq[occurrence.start():])
        else:
            continue

    output = []
    if len(final) >= 3:
        for i in [0, 1, 2]:
            output.append({'cq': final[i]})
        return output
    else:
        return 'Missing CQs'

In [8]:
def generate_critical_questions_batch(model, tokenizer, model_name, batch_data):
    prompts = [
        f"""Suggest 3 critical questions that should be raised before accepting the arguments in this text:\n\n\"{item['intervention']}\"\n\nGive one question per line. Make the questions simple, and do not give any explanation regarding why the question is relevant."""
        for item in batch_data
    ]
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)

    with torch.no_grad():
      outputs = model.generate(
          **inputs,
          max_new_tokens=512,
          do_sample=True,
          temperature=0.6,
          top_p=0.9
      )

    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    del inputs, outputs
    torch.cuda.empty_cache()

    return [
        structure_output(decoded[len(prompt):].strip())
        for decoded, prompt in zip(decoded_outputs, prompts)
    ]

In [10]:
with open(test_dataset_path, 'r') as f:
    data = json.load(f)


for model_info in models:
    logger.info(f"Loading model: {model_info['model_id']}")

    tokenizer = AutoTokenizer.from_pretrained(model_info["model_id"])
    if tokenizer.pad_token is None:
      tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        model_info["model_id"],
        torch_dtype=torch.bfloat16,
        device_map="auto"
    )

    output_data = {}
    items = list(data.items())

    for i in range(0, len(items), batch_size):
        batch = items[i:i+batch_size]
        batch_ids = [item_id for item_id, _ in batch]
        batch_data = [item for _, item in batch]

        questions_list = generate_critical_questions_batch(model, tokenizer, model_info["name"], batch_data)

        for item_id, questions in zip(batch_ids, questions_list):
            if questions == 'Missing CQs':
                questions = []

            output_data[item_id] = {
                "cqs": questions
            }

            logger.info(f"Generated {item_id}: {questions}")

    with open(model_info["output_file"], 'w') as f:
        json.dump(output_data, f, indent=2)

    logger.info(f"Output saved to {model_info['output_file']}")

INFO:__main__:Loading model: /content/drive/MyDrive/HSG/NLP/Project NLP/Models/Meta-Llama-3.1-8B-Instruct


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
INFO:__main__:Generated CLINTON_199_2: [{'cq': 'What specific actions has Donald Trump taken to demonstrate his dismissal of NATO?'}, {'cq': 'Can the information provided by Muslim communities be verified, or is it potentially biased?'}, {'cq': "Does the Clinton campaign's emphasis on Muslim communities' potential to provide intelligence reflect a broader strategy of using Islamophobia to gain political support? "}]
INFO:__main__:Generated CLINTON_1_2: [{'cq': 'Clinton proposes to address the stresses faced by working families?'}, {'cq': 'How does Clinton plan to ensure that the wealthy actually pay their fair share?'}, {'cq': 'What are the potential consequences of increasing taxes on corporations and the wealthy? '}]
INFO:__main__:Generated CLINTON_21: [{'cq': 'What are the sources of the statistics mentioned in the text?'}, {'cq': 'What are the specific policies proposed by Clinton and Trump that are being comp

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
INFO:__main__:Generated CLINTON_199_2: [{'cq': 'What specific intelligence benefits does Clinton claim will result from working more closely with allies?'}, {'cq': 'Does Clinton provide evidence for the effectiveness of working with Muslim-majority nations in combating terrorism?'}, {'cq': "How has Donald Trump's rhetoric specifically impacted Muslim communities and cooperation?"}]
INFO:__main__:Generated CLINTON_1_2: [{'cq': 'What specific policies does Clinton propose for supporting families?  '}, {'cq': 'How does Clinton plan to ensure the wealthy pay their fair share?  '}, {'cq': 'Are there any details on how closing corporate loopholes will fund her proposals?  '}]
INFO:__main__:Generated CLINTON_21: [{'cq': 'What evidence supports the claim that returning to previous policies would be detrimental?'}, {'cq': 'W

## Commit & Push

In [11]:
!git config --global user.name "Rico Städeli"
!git config --global user.email "rico@yabriga.ch"

In [12]:
commit_message = "Generate CQs for Baseline models"
!git add .
!git commit -m "{commit_message}"
!git push

[main 80a31ce] Generate CQs for Baseline models
 6 files changed, 212 insertions(+), 8252 deletions(-)
 delete mode 100644 Evaluation/Results/results_zeroshot_llama_3.1-8B-instruct-finetuned.json
 delete mode 100644 Evaluation/Results/results_zeroshot_llama_3.1-8B-instruct-finetuned_formated.json
 rewrite Evaluation/Results/results_zeroshot_llama_3.1-8B-instruct.json (99%)
 rewrite Evaluation/Results/results_zeroshot_qwen2.5-7b-instruction.json (99%)
 delete mode 100644 Logs/2_baseline_predictions.log
 delete mode 100644 Logs/3a_finetuned_cqs_generation.log
Enumerating objects: 13, done.
Counting objects: 100% (13/13), done.
Delta compression using up to 12 threads
Compressing objects: 100% (7/7), done.
Writing objects: 100% (7/7), 2.60 KiB | 2.60 MiB/s, done.
Total 7 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/RicoStaedeli/NLP2025_CQG.git
   c92d4b8..80a31ce  main -> main
